# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request
4. Create a basic AI agent
5. Handle basic error scenarios

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured

## Import Required Libraries and Setup

In the next cell, we'll:
1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard Python libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
   - This will automatically use your logged-in Azure CLI credentials
   - Alternatively, it can use other authentication methods like environment variables or managed identity


In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential, ChainedTokenCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials with tenant-specific authentication (same as environment setup)
from dotenv import load_dotenv
load_dotenv()

# Get the correct tenant ID from environment
correct_tenant_id = os.getenv("TENANT_ID")
print(f"🔑 Using Tenant ID: {correct_tenant_id}")

# Create a credential chain with tenant-specific authentication
def create_credential_chain_with_tenant():
    """Create a robust credential chain for authentication with specific tenant"""
    try:
        # Try Azure CLI first with the specific tenant
        cli_credential = AzureCliCredential(tenant_id=correct_tenant_id)
        
        # Create a chained credential with fallbacks, all using the correct tenant
        credential_chain = ChainedTokenCredential(
            cli_credential,
            InteractiveBrowserCredential(tenant_id=correct_tenant_id)
        )
        
        return credential_chain
    except Exception as e:
        print(f"⚠️ Credential chain creation error: {e}")
        # Fallback to DefaultAzureCredential with tenant specified
        return DefaultAzureCredential(tenant_id=correct_tenant_id)

# Initialize credentials
try:
    credential = create_credential_chain_with_tenant()
    
    # Test the credential by getting a token for the correct tenant
    test_token = credential.get_token("https://management.azure.com/.default")
    print("✅ Successfully initialized Azure credentials with correct tenant!")
    
except Exception as e:
    print(f"❌ Credential initialization failed: {str(e)}")
    print(f"💡 Please run the authentication fix from the environment setup notebook first")
    credential = None

🔑 Using Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
✅ Successfully initialized Azure credentials with correct tenant!


## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.example` file to `.env` from the root directory
> 2. Update the project endpoint in your `.env` file
> 3. Have a Foundry Project already provisioned in Azure AI Foundry

You can find your project endpoint in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="../images/foundry-endpoint.png" alt="Project Endpoint Location" width="75%"/>



## Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our `.env` file.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [2]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

try:
    # Get the project connection string (which is actually the endpoint URL)
    project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not project_endpoint:
        raise ValueError("AI_FOUNDRY_PROJECT_ENDPOINT not found in environment variables")

    print(f"🔗 Project Endpoint: {project_endpoint}")

    # Create AIProjectClient as done in environment_setup (consistent with other notebooks)
    client = AIProjectClient(
        credential=credential,
        endpoint=project_endpoint
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")
    print("💡 Tip: Make sure your AI_FOUNDRY_PROJECT_ENDPOINT is set in the .env file")

🔗 Project Endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
✓ Successfully initialized AIProjectClient


In [3]:
# Validate and clean the endpoint
print("🔍 Validating endpoint configuration...")

if project_endpoint and '#' in project_endpoint:
    print("⚠️  WARNING: Your endpoint contains a '#' character!")
    print(f"   Current value: {project_endpoint}")
    print("\n   This looks like a comment was left in your .env file.")
    
    # Extract the actual endpoint (everything before the #)
    clean_endpoint = project_endpoint.split('#')[0].strip()
    print(f"   Cleaned endpoint: {clean_endpoint}")
    
    print("\n🔧 To fix permanently:")
    print("   1. Open your .env file")
    print("   2. Find the AI_FOUNDRY_PROJECT_ENDPOINT line")
    print("   3. Remove everything after and including the '#'")
    print("   4. Save the file and reload this notebook")
    
    # Use the cleaned endpoint for this session
    project_endpoint = clean_endpoint
    print(f"\n✅ Using cleaned endpoint for this session: {project_endpoint}")
    
    # Recreate the client with the clean endpoint
    client = AIProjectClient(
        credential=credential,
        endpoint=project_endpoint
    )
    print("✓ Client recreated with cleaned endpoint")
else:
    print(f"✓ Endpoint looks clean: {project_endpoint}")

🔍 Validating endpoint configuration...
✓ Endpoint looks clean: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject


## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `.env` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [4]:
import os
from azure.ai.inference.models import UserMessage

print("🔍 Step 1: Loading environment variables...")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
print(f"✓ MODEL_DEPLOYMENT_NAME: {model_deployment_name}")

print("\n🔍 Step 2: Checking client object...")
print(f"✓ Client object exists: {client is not None}")
print(f"✓ Client type: {type(client)}")

try:
    print("\n🔍 Step 3: Getting OpenAI client...")
    aoai_client = client.get_openai_client(api_version="2024-10-21")
    print(f"✓ OpenAI client created: {type(aoai_client)}")
    
    print("\n🔍 Step 4: Creating chat completion...")
    response = aoai_client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {"role": "user", "content": "How to be healthy in one sentence?"}
        ]
    )
    print("✓ Chat completion successful!")
    
    print("\n🔍 Step 5: Extracting response...")
    print(f"✅ Response: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"\n❌ An error occurred at one of the above steps:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    print(f"\n💡 Tip: Make sure your MODEL_DEPLOYMENT_NAME ({model_deployment_name}) is correctly configured and deployed")
    import traceback
    print("\n🔍 Full traceback:")
    traceback.print_exc()


🔍 Step 1: Loading environment variables...
✓ MODEL_DEPLOYMENT_NAME: gpt-4o

🔍 Step 2: Checking client object...
✓ Client object exists: True
✓ Client type: <class 'azure.ai.projects._patch.AIProjectClient'>

🔍 Step 3: Getting OpenAI client...
✓ OpenAI client created: <class 'openai.lib.azure.AzureOpenAI'>

🔍 Step 4: Creating chat completion...
✓ Chat completion successful!

🔍 Step 5: Extracting response...
✅ Response: Maintain a balanced diet, exercise regularly, get adequate sleep, stay hydrated, manage stress, and avoid harmful substances.


## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure AI Agent Service, a powerful tool for building intelligent agents.

Azure AI Agent Service is a fully managed service that helps developers build, deploy, and scale AI agents
without managing infrastructure. It combines large language models with tools that allow agents to:
- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling 
- Automate complex workflows

The code below demonstrates how to:
1. Create an agent with a code interpreter tool
2. Create a conversation thread
3. Send a message requesting BMI analysis 
4. Process the request and get results
5. Save any generated visualizations to local files

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access
to a code interpreter tool for creating visualizations. This showcases how agents can combine
natural language understanding with computational capabilities.

> **Note:** Generated visualizations will be saved as PNG files in the same folder as this notebook.
 



In [ ]:
from azure.ai.agents.models import CodeInterpreterTool

print("🔍 Verifying Project Configuration...")
print(f"Project endpoint: {project_endpoint}")
print(f"Model deployment: {model_deployment_name}")

# Check if endpoint looks like an Azure AI Foundry endpoint
if "api.azureml.ms" in project_endpoint or "inference.ml.azure.com" in project_endpoint or "services.ai.azure.com" in project_endpoint:
    print("✓ Endpoint format looks correct for Azure AI Foundry")
elif "openai.azure.com" in project_endpoint:
    print("❌ This looks like an Azure OpenAI endpoint, not an Azure AI Foundry project endpoint")
    print("\n💡 To fix this:")
    print("1. Go to https://ai.azure.com")
    print("2. Select your project")
    print("3. Go to Settings → Project details")
    print("4. Copy the 'Project connection string' or 'Endpoint'")
    print("5. Update AI_FOUNDRY_PROJECT_ENDPOINT in your .env file")
    raise ValueError("Incorrect endpoint format - need Azure AI Foundry project endpoint")
else:
    print("⚠️ Unknown endpoint format")

print(f"Client type: {type(client)}")

# Additional diagnostics for agents API
print("\n🔍 Additional Diagnostics...")
try:
    # Try to get project properties to verify connectivity
    print("Testing project properties access...")
    # The client should have project properties
    if hasattr(client, 'project_url'):
        print(f"✓ Project URL: {client.project_url}")
    if hasattr(client, '_config'):
        print(f"✓ Client config available")
    
    # Check subscription and resource group info from endpoint
    import re
    endpoint_parts = project_endpoint.split('/')
    print(f"✓ Endpoint parts: {endpoint_parts}")
    
    # Try to extract project name from endpoint
    if '/projects/' in project_endpoint:
        project_name = project_endpoint.split('/projects/')[-1].split('/')[0]
        print(f"✓ Extracted project name: {project_name}")
    
except Exception as diag_error:
    print(f"⚠️ Diagnostic error: {diag_error}")

try:
    # Initialize the Code Interpreter Tool
    print("\n🔍 Step 1: Initializing Code Interpreter Tool...")
    code_interpreter = CodeInterpreterTool()
    print("✓ Code Interpreter Tool initialized")
    
    # Create an AI agent with the code interpreter tool
    print("\n🔍 Step 2: Creating AI agent...")
    print(f"Using model: {model_deployment_name}")
    print(f"Using endpoint: {project_endpoint}")
    
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions=(
            "You are a health analyst who calculates BMI using US metrics (pounds, feet/inches). "
            "Use average US female measurements: 5'4\" (69 inches) and 130 pounds. "
            "Create a visualization showing where this BMI falls on the scale."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"✓ Agent created with ID: {agent.id}")
    
    # Create a new conversation thread
    print("\n🔍 Step 3: Creating conversation thread...")
    thread = client.agents.threads.create()
    print(f"✓ Thread created with ID: {thread.id}")
    
    # Create a message requesting BMI analysis and visualization
    print("\n🔍 Step 4: Creating message...")
    message = client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=(
            "Calculate BMI for an average US female (5'4\", 130 lbs). "
            "Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35. "
            "Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization."
        )
    )
    print("✓ Message created")
    
    # Process the request by creating and running a thread run
    print("\n🔍 Step 5: Processing thread run...")
    run = client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print("✓ Thread run completed")
    
    # Get the agent's response
    print("\n🔍 Step 6: Getting agent response...")
    messages = client.agents.messages.list(thread_id=thread.id)
    
    # Print the assistant's response and save any images
    for message in messages:
        if message.role == "assistant":
            # Print text content
            for content_item in message.content:
                if hasattr(content_item, 'text'):
                    print(f"\n🤖 Assistant: {content_item.text.value}")
            
            # Save any image files using the correct API
            for img in message.image_contents:
                file_id = img.image_file.file_id
                file_name = f"bmi_analysis_{file_id}.png"
                
                try:
                    # Use the correct file saving method
                    client.agents.files.save(file_id=file_id, file_name=file_name)
                    print(f"📊 Visualization saved as: {file_name}")
                    
                except Exception as file_error:
                    print(f"⚠️ Could not save file {file_id}: {file_error}")
                    print("💡 You can view the file in the Azure AI Foundry portal")
            break
    
    # Cleanup by deleting the agent
    print("\n🔍 Step 7: Cleaning up...")
    client.agents.delete_agent(agent.id)
    print("✓ Agent deleted successfully")
    
except Exception as e:
    print(f"\n❌ An error occurred: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    
    if "ResourceNotFound" in str(type(e).__name__) or "does not exist" in str(e):
        print("\n🔧 Troubleshooting 'Project does not exist' error:")
        print("\n1️⃣ Verify your endpoint format in Azure AI Foundry:")
        print("   a. Go to https://ai.azure.com")
        print("   b. Select your project")
        print("   c. Go to Settings → Project details")
        print("   d. Look for 'Project connection string' - it should look like:")
        print("      https://<region>.services.ai.azure.com/discovery/projects/<subscription>/<rg>/<project-name>")
        print("   OR 'Discovery service endpoint':")
        print("      https://<resource-name>.services.ai.azure.com/api/projects/<project-name>")
        print(f"\n   Your current endpoint: {project_endpoint}")
        
        print("\n2️⃣ Common endpoint issues:")
        print("   • Make sure the endpoint includes the full project path")
        print("   • Verify the project name matches exactly (case-sensitive)")
        print("   • Check if you have access to the project in the portal")
        
        print("\n3️⃣ Verify Azure AI Agents availability:")
        print("   a. In Azure AI Foundry portal, check if 'Agents' appears in your project menu")
        print("   b. Your project region must support Agents (currently limited regions)")
        print("   c. Supported regions as of Dec 2024: East US 2, Sweden Central, West US 3")
        print(f"\n   Tip: Extract region from your endpoint: {project_endpoint.split('//')[1].split('.')[0]}")
        
        print("\n4️⃣ Try alternative endpoint format:")
        print("   If your endpoint is: https://X.services.ai.azure.com/api/projects/Y")
        print("   Try the discovery format: https://X.services.ai.azure.com/discovery/projects/<sub>/<rg>/Y")
        print("   (You'll need subscription ID and resource group name from Azure Portal)")
        
        print("\n5️⃣ Verify authentication and permissions:")
        print("   • Run: az account show  (to verify logged in to correct subscription)")
        print("   • Ensure you have 'Contributor' or 'Owner' role on the AI Foundry project")
        print("   • Try running: az login --tenant <your-tenant-id> to refresh credentials")
        
        print("\n📝 Note: If chat completions work but agents don't, this typically means:")
        print("   - Your project exists and credentials are fine")
        print("   - But the agents API endpoint routing needs adjustment")
        print("   - Or agents feature isn't available in your project's region")
    else:
        import traceback
        print("\n🔍 Full traceback:")
        traceback.print_exc()


🔍 Verifying Project Configuration...
Project endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
Model deployment: gpt-4o
✓ Endpoint format looks correct for Azure AI Foundry
Client type: <class 'azure.ai.projects._patch.AIProjectClient'>

🔍 Additional Diagnostics...
Testing project properties access...
✓ Client config available
✓ Endpoint parts: ['https:', '', 'demopocaifoundry.services.ai.azure.com', 'api', 'projects', 'demoproject']
✓ Extracted project name: demoproject

🔍 Step 1: Initializing Code Interpreter Tool...
✓ Code Interpreter Tool initialized

🔍 Step 2: Creating AI agent...
Using model: gpt-4o
Using endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
✓ Agent created with ID: asst_W5kkWllnJvKDo3M8IChjWpd0

🔍 Step 3: Creating conversation thread...
✓ Thread created with ID: thread_0deaZHuAAsjC0kbQk4RdFlH9

🔍 Step 4: Creating message...
✓ Message created

🔍 Step 5: Processing thread run...
✓ Thread run completed

🔍 

: 